In [8]:
import os
import sys
import urllib.request
import datetime
import time
import json

In [16]:
client_id = 'eG9_PaYuW0avp6Vmukzk' # 본인 id로 교체
client_secret = 'Gt6eBqEYSS'

In [17]:
#[CODE 1]
def getRequestUrl(url):    
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)
    
    try: 
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print ("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

In [18]:
#[CODE 2]
def getNaverSearch(node, srcText, start, display):    
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start, display)
    
    url = base + node + parameters    
    responseDecode = getRequestUrl(url)   #[CODE 1]
    
    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode)

In [19]:
#[CODE 3]
def getPostData(post, jsonResult, cnt):    
    title = post['title']
    description = post['description']
    org_link = post['originallink']
    link = post['link']
    
    pDate = datetime.datetime.strptime(post['pubDate'],  '%a, %d %b %Y %H:%M:%S +0900')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')
    
    jsonResult.append({'cnt':cnt, 'title':title, 'description': description, 
'org_link':org_link,   'link': org_link,   'pDate':pDate})
    return    

In [24]:
#[CODE 0]
def main():
    node = 'news'   # 크롤링 할 대상
    srcText = input('검색어를 입력하세요: ')
    cnt = 0
    jsonResult = []

    jsonResponse = getNaverSearch(node, srcText, 1, 100)  #[CODE 2]
    total = jsonResponse['total']
 
    while ((jsonResponse != None) and (jsonResponse['display'] != 0)):         
        for post in jsonResponse['items']:
            cnt += 1
            #print(post)
            getPostData(post, jsonResult, cnt)  #[CODE 3]       
        
        start = jsonResponse['start'] + jsonResponse['display']
        jsonResponse = getNaverSearch(node, srcText, start, 100)  #[CODE 2]
       
    print('전체 검색 : %d 건' %total)
    
    with open('%s_naver_%s.json' % (srcText, node), 'w', encoding='utf8') as outfile:
        jsonFile = json.dumps(jsonResult,  indent=4, sort_keys=True,  ensure_ascii=False)
                        
        outfile.write(jsonFile)
        
    print("가져온 데이터 : %d 건" %(cnt))
    print ('%s_naver_%s.json SAVED' % (srcText, node))

In [28]:
if __name__ == '__main__':
    main()

검색어를 입력하세요: 백신
[2021-10-28 15:20:05.497259] Url Request Success
[2021-10-28 15:20:06.611535] Url Request Success
[2021-10-28 15:20:07.749042] Url Request Success
[2021-10-28 15:20:08.865117] Url Request Success
[2021-10-28 15:20:10.179188] Url Request Success
[2021-10-28 15:20:11.310525] Url Request Success
[2021-10-28 15:20:12.412643] Url Request Success
[2021-10-28 15:20:13.519542] Url Request Success
[2021-10-28 15:20:14.616873] Url Request Success
[2021-10-28 15:20:15.922532] Url Request Success
HTTP Error 400: Bad Request
[2021-10-28 15:20:16.774185] Error for URL : https://openapi.naver.com/v1/search/news.json?query=%EB%B0%B1%EC%8B%A0&start=1001&display=100
전체 검색 : 1298004 건
가져온 데이터 : 1000 건
백신_naver_news.json SAVED
